In [3]:
import pandas as pd

pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_irradiance.csv', parse_dates=True, index_col='timeStamp')

,ghi,dni,dhi
timeStamp,,,
2014-01-02 08:00:00,0.0,0.0,0.0
2014-01-02 08:01:00,0.0,0.0,0.0
2014-01-02 08:02:00,0.0,0.0,0.0
2014-01-02 08:03:00,0.0,0.0,0.0
2014-01-02 08:04:00,0.0,0.0,0.0
...,...,...,...
2016-12-31 07:55:00,0.0,0.0,0.0
2016-12-31 07:56:00,0.0,0.0,0.0
2016-12-31 07:57:00,0.0,0.0,0.0


In [8]:
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.579454_lon-121.260320.csv', parse_dates=True, skiprows=1, index_col='reftime')

,valtime,dwsw,cloud_cover,precipitation,pressure,wind-u,wind-v,temperature,rel_humidity
reftime,,,,,,,,,
2014-01-01 12:00:00,2014-01-02 14:00:00,0.000,2.0,0.000,101560.0,-1.2019,-0.8152,273.254,53.0
2014-01-01 12:00:00,2014-01-02 15:00:00,0.000,0.0,0.000,101546.0,-1.0587,-0.6262,272.865,53.0
2014-01-01 12:00:00,2014-01-02 16:00:00,95.000,0.0,0.000,101576.0,-0.8595,-0.3518,276.175,52.0
2014-01-01 12:00:00,2014-01-02 17:00:00,270.875,0.0,0.000,101649.0,-0.4019,-0.0743,283.253,46.0
2014-01-01 12:00:00,2014-01-02 18:00:00,420.750,16.0,0.000,101657.0,0.4367,-0.5003,288.350,40.0
...,...,...,...,...,...,...,...,...,...
2016-12-31 12:00:00,2017-01-01 23:00:00,98.250,100.0,0.375,100790.0,1.0159,-0.2391,281.583,81.0
2016-12-31 12:00:00,2017-01-02 00:00:00,40.125,100.0,0.375,100810.0,1.1071,0.3261,280.958,81.0
2016-12-31 12:00:00,2017-01-02 03:00:00,0.000,100.0,0.000,100855.0,0.0090,1.8306,279.198,90.0


In [10]:
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.579454_lon-121.260320.csv', parse_dates=True, skiprows=1, index_col='reftime')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.599891_lon-121.126680.csv', parse_dates=True, skiprows=1, index_col='reftime')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.683880_lon-121.286556.csv', parse_dates=True, skiprows=1, index_col='reftime')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_NAM_lat38.704328_lon-121.152788.csv', parse_dates=True, skiprows=1, index_col='reftime')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_irradiance.csv', parse_dates=True,index_col='timeStamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_satellite.csv', parse_dates=True)
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_sky_image_features.csv', parse_dates=True,index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Folsom_weather.csv', parse_dates=True, index_col='timeStamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Irradiance_features_day-ahead.csv', parse_dates=True,index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Irradiance_features_intra-day.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Irradiance_features_intra-hour.csv', parse_dates=True,  index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/NAM_nearest_node_day-ahead.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Sat_image_features_intra-day.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Sky_image_features_intra-hour.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Target_intra-day.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Target_day-ahead.csv', parse_dates=True, index_col='timestamp')
pd.read_csv('/Users/adam/Documents/Projects/irradianceforecasting/data/raw/Target_intra-hour.csv', parse_dates=True, index_col='timestamp')


,ghi_5min,dni_5min,ghi_clear_5min,dni_clear_5min,ghi_kt_5min,dni_kt_5min,elevation_5min,ghi_10min,dni_10min,ghi_clear_10min,...,ghi_kt_25min,dni_kt_25min,elevation_25min,ghi_30min,dni_30min,ghi_clear_30min,dni_clear_30min,ghi_kt_30min,dni_kt_30min,elevation_30min
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-01-02 16:00:00,74.342,507.160,45.481545,236.113747,1.200000,1.200000,5.821237,89.860,559.840,56.989441,...,1.200000,1.200000,8.927891,143.860,639.280,107.442565,418.638252,1.2,1.2,9.681759
2014-01-02 16:05:00,89.860,559.840,56.989441,277.436445,1.200000,1.200000,6.611024,105.300,599.620,69.080774,...,1.200000,1.200000,9.681759,156.800,652.760,120.585989,447.786314,1.2,1.2,10.425939
2014-01-02 16:10:00,105.300,599.620,69.080774,316.517024,1.200000,1.200000,7.392244,118.220,614.020,81.599285,...,1.200000,1.200000,10.425939,169.880,665.500,133.784150,474.715601,1.2,1.2,11.160135
2014-01-02 16:15:00,118.220,614.020,81.599285,353.096042,1.200000,1.200000,8.164625,129.880,619.940,94.419980,...,1.200000,1.200000,11.160135,186.780,693.040,146.982563,499.592048,1.2,1.2,11.884044
2014-01-02 16:20:00,129.880,619.940,94.419980,387.116288,1.200000,1.200000,8.927891,143.860,639.280,107.442565,...,1.200000,1.200000,11.884044,201.400,712.140,160.135837,522.580864,1.2,1.2,12.597357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-30 23:55:00,109.300,401.520,79.738806,347.879668,1.200000,1.154725,8.051949,112.040,376.340,67.333420,...,1.132514,0.798178,4.919444,32.594,24.764,23.905552,144.184703,1.2,1.2,4.115458
2016-12-31 00:00:00,112.040,376.340,67.333420,311.112695,1.200000,1.192924,7.281865,79.014,221.200,55.366935,...,1.200000,1.200000,4.115458,26.084,8.678,15.699923,102.362578,1.2,1.2,3.303639
2016-12-31 00:05:00,79.014,221.200,55.366935,271.872783,1.150846,0.802914,6.502907,45.668,86.552,44.000729,...,1.200000,1.200000,3.303639,22.898,3.130,9.111790,64.742781,1.2,1.2,2.484232
